In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Cleaning

##### Reading the csv file

In [4]:
# Reading a csv file
trargon=pd.read_csv('Traralgon.csv')
trargon.head()

,id,name,month,average,measurement_count,parameter,parameterId,displayName,unit
0,10771,Traralgon,1/1/2020,47.228080,578,pm10,1,PM10,µg/m³
1,10771,Traralgon,1/1/2020,26.981900,542,pm25,2,PM2.5,µg/m³
2,10771,Traralgon,1/1/2020,0.006017,556,no2,7,NO₂,ppm
3,10771,Traralgon,1/1/2020,0.498507,556,co,8,CO,ppm
4,10771,Traralgon,1/1/2020,0.000784,496,so2,9,SO₂,ppm


##### Here we are dropping the columns that we don't need for the analysis

In [5]:
# Dropping unnecessary columns
trargon=trargon.drop(['measurement_count', 'parameterId','displayName'], axis=1)
# Checking the first few rows
trargon.head()

,id,name,month,average,parameter,unit
0,10771,Traralgon,1/1/2020,47.228080,pm10,µg/m³
1,10771,Traralgon,1/1/2020,26.981900,pm25,µg/m³
2,10771,Traralgon,1/1/2020,0.006017,no2,ppm
3,10771,Traralgon,1/1/2020,0.498507,co,ppm
4,10771,Traralgon,1/1/2020,0.000784,so2,ppm


##### Checking for duplicate values

In [6]:
# Check for duplicated rows
duplicates = trargon[trargon.duplicated(keep=False)]

# Display the duplicated rows
print(duplicates)

Empty DataFrame
Columns: [id, name, month, average, parameter, unit]
Index: []


##### Modifying the months column

In [7]:
# Creating a dataframe
trargon = pd.DataFrame(trargon)

# Convert the 'month' column to datetime
trargon['month']=pd.to_datetime(trargon['month'])

# Apply strftime to the 'month' column to format it as "Month Year"
trargon['formatted_month'] = trargon['month'].dt.strftime('%B %Y')

# Checking on few rows
trargon.head()

,id,name,month,average,parameter,unit,formatted_month
0,10771,Traralgon,2020-01-01,47.228080,pm10,µg/m³,January 2020
1,10771,Traralgon,2020-01-01,26.981900,pm25,µg/m³,January 2020
2,10771,Traralgon,2020-01-01,0.006017,no2,ppm,January 2020
3,10771,Traralgon,2020-01-01,0.498507,co,ppm,January 2020
4,10771,Traralgon,2020-01-01,0.000784,so2,ppm,January 2020


In [8]:
# Removing the previous 'month' column
trargon=trargon.drop('month',axis=1)
trargon.head()
  

,id,name,average,parameter,unit,formatted_month
0,10771,Traralgon,47.228080,pm10,µg/m³,January 2020
1,10771,Traralgon,26.981900,pm25,µg/m³,January 2020
2,10771,Traralgon,0.006017,no2,ppm,January 2020
3,10771,Traralgon,0.498507,co,ppm,January 2020
4,10771,Traralgon,0.000784,so2,ppm,January 2020


In [9]:
# Splitting the formatted_month column to month and year
trargon[['months','year']] = trargon['formatted_month'].str.split(' ',expand=True)
trargon 

# Removing the formatted_month column
trargon = trargon.drop('formatted_month',axis=1)
trargon.head()

# Export the DataFrame to a CSV file
csv_file_path = 'trargon_modified.csv'
trargon.to_csv(csv_file_path, index=False)

#### Origanizing the dataframe in a way that the parameters are the columns

In [10]:
# Step 1: Assess data completeness
completeness = trargon.groupby(['name', 'parameter'])['average'].count().unstack()
print("Data completeness (number of measurements):")
print(completeness)

# Calculate the percentage of missing data for each city-parameter combination
missing_percentage = (1 - completeness.divide(completeness.max())) * 100
print("\nPercentage of missing data:")
print(missing_percentage)

# Step 2: Filter out parameters or trargon with too much missing data
threshold = 50
valid_parameters = missing_percentage.columns[missing_percentage.mean() < threshold]
valid_trargon = missing_percentage.index[missing_percentage.mean(axis=1) < threshold]

# Filter the original dataset
trargon_filtered = trargon[
    (trargon['name'].isin(valid_trargon)) & 
    (trargon['parameter'].isin(valid_parameters))
]

# Step 3: Handle remaining missing values
trargon_pivoted = trargon_filtered.pivot_table(
    index=['name', 'months', 'year'],
    columns='parameter',
    values='average'
).reset_index()

trargon_cleaned = trargon_pivoted.groupby('name').apply(
    lambda x: x.sort_values(['year', 'months']).ffill()
).reset_index(drop=True)

# Convert 'months' to datetime for proper sorting
trargon_cleaned['date'] = pd.to_datetime(trargon_cleaned['year'].astype(str) + ' ' + trargon_cleaned['months'], format='%Y %B')
trargon_cleaned = trargon_cleaned.sort_values(['name', 'date'])

print("\nCleaned data shape:", trargon_cleaned.shape)
print(trargon_cleaned.head())

# Get the list of parameters
parameters = trargon_cleaned.columns.drop(['name', 'months', 'year', 'date'])

# Export the resulting DataFrame to a CSV file
trargon_cleaned.to_csv('cleaned_trargon_data.csv', index=False)

Data completeness (number of measurements):
parameter  co  no2  o3  pm10  pm25  so2
name                                   
Traralgon  17   17  16    17    17   16

Percentage of missing data:
parameter   co  no2   o3  pm10  pm25  so2
name                                     
Traralgon  0.0  0.0  0.0   0.0   0.0  0.0

Cleaned data shape: (17, 10)
parameter       name    months  year        co       no2        o3       pm10  \
4          Traralgon   January  2020  0.498507  0.006017  0.022323  47.228080   
3          Traralgon  February  2020  0.126760  0.004099  0.015773  16.824422   
7          Traralgon     March  2020  0.101695  0.005530  0.015459  17.395365   
0          Traralgon     April  2020  0.105932  0.005480  0.015225  14.437795   
8          Traralgon       May  2020  0.351573  0.007809  0.011008  14.854943   

parameter       pm25       so2       date  
4          26.981900  0.000784 2020-01-01  
3           6.648671  0.000852 2020-02-01  
7           5.900118  0.000402 2

C:\Users\admin\AppData\Local\Temp\ipykernel_15664\2479337250.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  trargon_cleaned = trargon_pivoted.groupby('name').apply(


#### Unit Conversion

In [11]:
# Load the dataset
file_path = 'cleaned_trargon_data.csv'  
df = pd.read_csv(file_path)

# Conversion factors (from ppm to µg/m³)
molar_masses = {
    'no2': 46.01,  # g/mol for NO2
    'so2': 64.07,  # g/mol for SO2
    'co': 28.01,   # g/mol for CO
    'o3': 48.00    # g/mol for O3
}

# Standard volume at STP in liters
V_m = 22.414

# Convert from ppm to µg/m³
for gas in molar_masses.keys():
    df[gas] = df[gas] * molar_masses[gas] * 1000 / V_m

# Save the converted dataset
df.to_csv('converted_trargon_city_data.csv', index=False)

print("Conversion completed and saved as 'converted_trargon_city_data.csv'")


Conversion completed and saved as 'converted_trargon_city_data.csv'


##### Calculating AQI

In [12]:
def calculate_aqi(concentration, breakpoints):
    for low, high, i_low, i_high in breakpoints:
        if low <= concentration <= high:
            return ((i_high - i_low) / (high - low)) * (concentration - low) + i_low
    return None

def calculate_overall_aqi(row, breakpoints):
    aqi_values = []
    for pollutant in ['pm25', 'pm10', 'o3', 'no2','so2','co']:
        if pd.notnull(row[pollutant]):
            aqi = calculate_aqi(row[pollutant], breakpoints[pollutant])
            if aqi is not None:
                aqi_values.append(aqi)
    return max(aqi_values) if aqi_values else None

# Load your CSV file
realD = pd.read_csv('converted_trargon_city_data.csv') 

# Define the breakpoints
# Define the breakpoints
breakpoints = {
    'pm10': [
        (0, 54, 0, 50),(55, 154, 51, 100),(155, 254, 101, 150),(255, 354, 151, 200),
        (355, 424, 201, 300),(425, 504, 301, 400),(505, 604, 401, 500)],
    'pm25': [
        (0.0, 12.0, 0, 50),(12.1, 35.4, 51, 100),(35.5, 55.4, 101, 150),(55.5, 150.4, 151, 200),
        (150.5, 250.4, 201, 300), (250.5, 350.4, 301, 400), (350.5, 500.4, 401, 500)],
    'no2': [
        (0, 53, 0, 50),(54, 100, 51, 100), (101, 360, 101, 150), (361, 649, 151, 200),
        (650, 1249, 201, 300),(1250, 1649, 301, 400),(1650, 2049, 401, 500)],
    'co': [ (0.0, 4.4, 0, 50), (4.5, 9.4, 51, 100), (9.5, 12.4, 101, 150),
        (12.5, 15.4, 151, 200),(15.5, 30.4, 201, 300),(30.5, 40.4, 301, 400),(40.5, 50.4, 401, 500)],
    'so2': [ (0, 35, 0, 50),(36, 75, 51, 100),(76, 185, 101, 150),(186, 304, 151, 200),
        (305, 604, 201, 300),(605, 804, 301, 400),(805, 1004, 401, 500)],
    'o3': [
        (0, 54, 0, 50),(55, 70, 51, 100),(71, 85, 101, 150),(86, 105, 151, 200),(106, 200, 201, 300),(201, 504, 301, 500)
    ]}


# Calculate AQI for each pollutant and overall AQI
for pollutant in ['pm25', 'pm10', 'o3', 'no2','so2','co']:
    realD[f'{pollutant}_aqi'] = realD[pollutant].apply(lambda x: calculate_aqi(x, breakpoints[pollutant]))

realD['overall_aqi'] = realD.apply(lambda row: calculate_overall_aqi(row, breakpoints), axis=1)

# Save the results
realD.to_csv('trargon_aqi_results.csv', index=False)

print(realD.head())  # Display the first few rows of the results

        name    months  year          co        no2         o3       pm10  \
0  Traralgon   January  2020  622.967186  12.350339  47.805706  47.228080   
1  Traralgon  February  2020  158.407306   8.413713  33.778020  16.824422   
2  Traralgon     March  2020  127.084292  11.351874  33.106339  17.395365   
3  Traralgon     April  2020  132.379429  11.249090  32.605511  14.437795   
4  Traralgon       May  2020  439.348652  16.029719  23.572840  14.854943   

        pm25       so2        date   pm25_aqi   pm10_aqi     o3_aqi  \
0  26.981900  2.240912  2020-01-01  82.296700  43.729703  44.264543   
1   6.648671  2.434857  2020-02-01  27.702797  15.578168  31.275945   
2   5.900118  1.148463  2020-03-01  24.583826  16.106820  30.654018   
3   7.421554  1.818709  2020-04-01  30.923144  13.368329  30.190288   
4  10.833890  1.445605  2020-05-01  45.141207  13.754577  21.826704   

     no2_aqi   so2_aqi co_aqi  overall_aqi  
0  11.651263  3.201303   None    82.296700  
1   7.937465  3.4783

##### Rounding the values of the AQI

In [13]:
# Drop specified columns
rnew= pd.read_csv('trargon_aqi_results.csv')
rnew.drop(columns=['pm10', 'pm25','no2','o3','so2','co','co_aqi'], inplace=True, errors='ignore')

# Round the average column
rnew[['pm25_aqi','pm10_aqi','o3_aqi','no2_aqi','so2_aqi','overall_aqi']] = rnew[['pm25_aqi','pm10_aqi','o3_aqi','no2_aqi','so2_aqi','overall_aqi']].round()

# Export the resulting DataFrame to a CSV file
rnew.to_csv('trargon_aqii_results.csv', index=False)
rnew.head()

,name,months,year,date,pm25_aqi,pm10_aqi,o3_aqi,no2_aqi,so2_aqi,overall_aqi
0,Traralgon,January,2020,2020-01-01,82.0,44.0,44.0,12.0,3.0,82.0
1,Traralgon,February,2020,2020-02-01,28.0,16.0,31.0,8.0,3.0,31.0
2,Traralgon,March,2020,2020-03-01,25.0,16.0,31.0,11.0,2.0,31.0
3,Traralgon,April,2020,2020-04-01,31.0,13.0,30.0,11.0,3.0,31.0
4,Traralgon,May,2020,2020-05-01,45.0,14.0,22.0,15.0,2.0,45.0
